In [3]:
import xarray as xr
import sys

rte_rrtmgp_dir = "../../rrtmgp-data"
clear_sky_example_files = f"{rte_rrtmgp_dir}/examples/rfmip-clear-sky/inputs"

rfmip = xr.load_dataset(
    f"{clear_sky_example_files}/multiple_input4MIPs_radiation_RFMIP_UColorado-RFMIP-1-2_none.nc"
)

rfmip = rfmip.sel(expt=0)  # only one experiment
kdist = xr.load_dataset(f"{rte_rrtmgp_dir}/rrtmgp-gas-lw-g256.nc")

# RRTMGP won't run with pressure less than its minimum. so we add a small value to the minimum pressure
press_min = rfmip["pres_level"].min()
min_index = rfmip["pres_level"].argmin()
rfmip["pres_level"][min_index] = press_min + sys.float_info.epsilon

kdist_gas_names = [n.decode().strip() for n in kdist["gas_names"].values]
rfmip_vars = list(rfmip.keys())

gas_names = {n: n + "_GM" for n in kdist_gas_names if n + "_GM" in rfmip_vars}

# Create a dict that maps the gas names in the kdist gas names to the gas names in the rfmip dataset
gas_names.update(
    {
        "co": "carbon_monoxide_GM",
        "ch4": "methane_GM",
        "o2": "oxygen_GM",
        "n2o": "nitrous_oxide_GM",
        "n2": "nitrogen_GM",
        "co2": "carbon_dioxide_GM",
        "ccl4": "carbon_tetrachloride_GM",
        "cfc22": "hcfc22_GM",
        "h2o": "water_vapor",
        "o3": "ozone",
    }
)

In [4]:
from pyrte_rrtmgp.rrtmgp import interpolation
from pyrte_rrtmgp.utils import rfmip_2_col_gas, flavors_from_kdist

neta = len(kdist["mixing_fraction"])
press_ref = kdist["press_ref"].values
temp_ref = kdist["temp_ref"].values
tlay = rfmip["temp_layer"].values
play = rfmip["pres_layer"].values
press_ref_trop = kdist["press_ref_trop"].values.item()

# start on 1 as we ignore dry air that is 0
vmr_idx = [i for i, g in enumerate(kdist_gas_names, 1) if g in gas_names]
vmr_ref = kdist["vmr_ref"].sel(absorber_ext=vmr_idx).values.transpose(2, 1, 0)

col_gas = rfmip_2_col_gas(rfmip, gas_names.values(), dry_air=False)
flavor = flavors_from_kdist(kdist)

jtemp, fmajor, fminor, col_mix, tropo, jeta, jpress = interpolation(
    neta=neta,
    flavor=flavor,
    press_ref=press_ref,
    temp_ref=temp_ref,
    press_ref_trop=press_ref_trop,
    vmr_ref=vmr_ref,
    play=play,
    tlay=tlay,
    col_gas=col_gas,
)

In [ ]:
from pyrte_rrtmgp.rrtmgp import rrtmgp_compute_Planck_source
from pyrte_rrtmgp.utils import rfmip_2_col_gas, flavors_from_kdist

